In [132]:
import warnings
warnings.filterwarnings('ignore')

In [133]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("muhammadshahidazeem/customer-churn-dataset")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\User\.cache\kagglehub\datasets\muhammadshahidazeem\customer-churn-dataset\versions\1


In [134]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [135]:
df=pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [136]:
df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [137]:
# preprocess data

# drop irrelevant feature
ls=['RowNumber','CustomerId','Surname']
df=df.drop(columns=ls,axis=1)

In [138]:
df.columns

Index(['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary',
       'Exited'],
      dtype='object')

In [139]:
# Encode cat variables

label_encoder=LabelEncoder()
df['Gender']=label_encoder.fit_transform(df['Gender'])

df['Gender'].unique()

array([0, 1])

In [140]:
df['Geography'].unique() # We will do onehotencoding

array(['France', 'Spain', 'Germany'], dtype=object)

In [141]:
## OneHotEncode the geography

from sklearn.preprocessing import OneHotEncoder
onehotencode=OneHotEncoder()
geo_enc=onehotencode.fit_transform(df[['Geography']])

geo_enc

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [142]:
geo_features=onehotencode.get_feature_names_out()
geo_features

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [143]:
geo_array=geo_enc.toarray()
geo_array

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]], shape=(10000, 3))

In [144]:
geo_df=pd.DataFrame(geo_array,columns=geo_features)
geo_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [145]:
# Concat with the original dataframe

df=pd.concat([df.drop('Geography',axis=1),geo_df],axis=1)

In [146]:
df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [147]:
## Saving the trained encoders as a file

with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder,file)

with open('onehotencoder_geography.pkl','wb') as file:
    pickle.dump(onehotencode,file)

In [148]:
# divide dataset into independent and dependent dataset

x=df.drop(columns=['Exited'],axis=1)
y=df['Exited']

In [149]:
x

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,0.0,1.0,0.0


In [150]:
y

0       1
1       0
2       1
3       0
4       0
       ..
9995    0
9996    0
9997    1
9998    1
9999    0
Name: Exited, Length: 10000, dtype: int64

In [151]:
# Splitting

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [152]:
df.shape,x_train.shape,x_test.shape

((10000, 13), (8000, 12), (2000, 12))

In [153]:
# scaling

sc=StandardScaler()
x_train_scaled=sc.fit_transform(x_train)
x_test_scaled=sc.transform(x_test)

In [154]:
# saving the scaler file

with open('scaler.pkl','wb') as file:
    pickle.dump(sc,file)

## Training With ANN With Optimizer and Loss Functions

In [155]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [156]:
(x_train_scaled.shape[1],)

(12,)

In [157]:
# Building an ANN: 

model=Sequential(
    [
        Dense(64, activation='relu',input_shape=(x_train_scaled.shape[1],)), # HL1 connected with the input layer using the input_shape() parameter
        Dense(32, activation='relu'),
        Dense(1,activation='sigmoid') # output layer
    ]
)

In [158]:
model.summary() # Number of parameters to train 

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [159]:
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.BinaryCrossentropy()
loss

<LossFunctionWrapper(<function binary_crossentropy at 0x00000234FAE6A8C0>, kwargs={'from_logits': False, 'label_smoothing': 0.0, 'axis': -1})>

In [160]:
# Compile the model

model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])

In [161]:
# Set up the tensorboard before training

log_dir="logs/fit" + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tensorflow_callback= TensorBoard(log_dir=log_dir,histogram_freq=1)

In [162]:
## Set up the Early Stopping- to stop the training midway in between
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True) # restore_best_weight is restore the best weight of all the epochs

In [163]:
## Train the model
history=model.fit(
    x_train_scaled,y_train,validation_data=(x_test_scaled,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.8354 - loss: 0.3964 - val_accuracy: 0.8575 - val_loss: 0.3517
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8531 - loss: 0.3542 - val_accuracy: 0.8590 - val_loss: 0.3498
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8586 - loss: 0.3493 - val_accuracy: 0.8610 - val_loss: 0.3441
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8587 - loss: 0.3427 - val_accuracy: 0.8560 - val_loss: 0.3383
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8609 - loss: 0.3371 - val_accuracy: 0.8560 - val_loss: 0.3395
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8619 - loss: 0.3364 - val_accuracy: 0.8585 - val_loss: 0.3437
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8615 - loss: 0.3338 - val_accuracy: 0.8555 - val_loss: 0.3372
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8643 - loss: 0.3320 - val_accu

In [164]:
model.save('model.h5')

In [165]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [166]:
%tensorboard --logdir logs/fit20251214-223502

Reusing TensorBoard on port 6007 (pid 12768), started 12:17:28 ago. (Use '!kill 12768' to kill it.)

Prediction

In [167]:
from tensorflow.keras.models import load_model

model=load_model('model.h5')

with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender=pickle.load(file)

with open('scaler.pkl','rb') as file:
    sc=pickle.load(file)

with open('onehotencoder_geography.pkl','rb') as file:
    onehotencoder_geography=pickle.load(file)

In [168]:
## Predicting the unseen value

input_data={
    'CreditScore':600,
    'Gender':'Male',
    'Geography':'France',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}

In [169]:
input_data_df=pd.DataFrame([input_data])
input_data_df

,CreditScore,Gender,Geography,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,Male,France,40,3,60000,2,1,1,50000


In [170]:
geo_encoded=onehotencoder_geography.transform([[input_data['Geography']]]).toarray()

geo_encoded_df=pd.DataFrame(geo_encoded,columns=onehotencoder_geography.get_feature_names_out(['Geography']))

geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [171]:
input_data_df=pd.concat([input_data_df.reset_index(drop=True), geo_encoded_df], axis=1)


In [172]:
input_data_df

,CreditScore,Gender,Geography,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,Male,France,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [173]:
## Encoding the gender
input_data_df['Gender']=label_encoder.transform(input_data_df['Gender'])

In [ ]:
input_data_df=input_data_df.drop(columns=['Geography'])

In [177]:
input_data_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [179]:
## Scaling the data
input_data_df_scaled=sc.transform(input_data_df)
input_data_df_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [ ]:
prediction=model.predict(input_data_df_scaled)
prediction # it returns the 2D array

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 616ms/step


array([[0.02911222]], dtype=float32)

In [185]:
if prediction[0][0] >= 0.5:
    print('Customer is likely to exit...')
else:
    print('Customer is likely to stay...')

Customer is likely to stay...
